In [1]:
import tifffile
import numpy as np
from matplotlib import colors
import matplotlib.pyplot as plt
from skimage import segmentation
from scipy.ndimage import binary_dilation
from matplotlib.colors import ListedColormap
from matplotlib_scalebar.scalebar import ScaleBar
import anndata as ad
import json
import os

In [2]:
def process_full_image(cell_df, seg_mask_path, target_col, label_col='label'):
    segmentation_mask = tifffile.imread(seg_mask_path) # read the whole segmentation mask - adapt code if lazy loading is needed
    mask = np.isin(segmentation_mask, cell_df[label_col].values) # filter out cells not in the cell dataframe based on cell id, e.g. if removing artefacts
    segmentation_mask[~mask] = 0

    plot_segmentation = np.zeros(segmentation_mask.shape, dtype=np.uint8) # create an empty array for plotting - this array will be filled with cell type labels
    unique_identifiers = set(cell_df[target_col].unique())
    for i, identifier in enumerate(unique_identifiers):
        print(identifier, i+1)
        layer_labels = cell_df.loc[cell_df[target_col] == identifier, label_col] # Thanks Lukas for catching this
        plot_segmentation[np.isin(segmentation_mask, layer_labels)] = i+1
    
    return segmentation_mask, plot_segmentation

def crop_roi(segmentation_mask, plot_segmentation, ROI_upper_left, ROI_lower_right):
    roi_crop = segmentation_mask[ ROI_upper_left[0]:ROI_lower_right[0], ROI_upper_left[1]:ROI_lower_right[1]]
    plot_crop = plot_segmentation[ROI_upper_left[0]:ROI_lower_right[0], ROI_upper_left[1]:ROI_lower_right[1]]
    return roi_crop, plot_crop

def create_plot(roi_crop, plot_crop, cmap, boundaries=False, boundary_color=(0, 0, 0, 1), boundary_thickness=1,save_path=None):
    plt.figure(figsize=(10, 10))
    plt.gca().set_aspect('equal', adjustable='box')
    scalebar = ScaleBar(1, 'um', location='lower left')
    plt.gca().add_artist(scalebar)
    plt.imshow(plot_crop, cmap=cmap, interpolation='none')
    if boundaries:
        boundaries_prediction = segmentation.find_boundaries(roi_crop, mode='thick')
        for _ in range(boundary_thickness - 1):  # Apply dilation multiple times for thicker boundaries
            boundaries_prediction = binary_dilation(boundaries_prediction)
        # Create an RGBA overlay: boundary pixels get the specified color, others are transparent
        boundary_overlay = np.zeros((*boundaries_prediction.shape, 4))  # Initialize as transparent
        boundary_overlay[boundaries_prediction] = boundary_color  # Assign color where boundary is detected

        # Overlay the boundaries while keeping the original image
        plt.imshow(boundary_overlay, interpolation='none')
    plt.axis('off')
    if save_path:
        plt.savefig(save_path, bbox_inches='tight', pad_inches=0)


In [3]:
extension = 'svg'
save_path = f'/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/paper/plots/figures/figure_1/{extension}/'
save_path_supp = f'/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/paper/plots/figures/suppl_figure_1/{extension}/'
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
adata = ad.read_h5ad("/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/standard/adatas/cells_final.h5ad")
with open('/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/github/myeloma_standal/src/Paper/figure_plots/neighborhood_color_map.json', 'r') as f:
    neighborhood_color_map = json.load(f)
with open('/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/github/myeloma_standal/src/Paper/figure_plots/phenotype_color_map.json', 'r') as f:
    phenotype_color_map = json.load(f)
with open('/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/github/myeloma_standal/src/Paper/figure_plots/disease_color_map.json', 'r') as f:
    disease_color_map = json.load(f)
with open('/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/github/myeloma_standal/src/Paper/figure_plots/disease3_color_map.json', 'r') as f:
    disease3_color_map = json.load(f)
neighborhood_colors = [neighborhood_color_map[cat] for cat in list(adata.obs['cellcharter_CN'].cat.categories)]
phenotype_colors = [phenotype_color_map[cat] for cat in list(adata.obs['Phenotype4'].cat.categories)]
disease_colors = [disease_color_map[cat] for cat in list(adata.obs['disease2'].cat.categories)]
disease3_colors = [disease3_color_map[cat] for cat in list(adata.obs['disease3'].cat.categories)]

In [ ]:
df = adata.to_df()
cell_df_whole = df.join(adata.obs)
sample = 'TS-373_IMC42_B_001.csv'
seg_name = sample.split('.csv')[0] + '.tiff'
seg_mask_path = f'/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/standard/masks/{seg_name}'
cell_df = cell_df_whole.loc[cell_df_whole['image_ID'] == sample]
cell_df = cell_df[cell_df['Phenotype4'] != 'exclude'] 

In [ ]:
target_col = 'cellcharter_CN' # column of interest for color coding
unique_identifiers = set(cell_df[target_col].unique())
segmentation_mask, plot_segmentation = process_full_image(cell_df=cell_df, seg_mask_path=seg_mask_path, target_col=target_col, label_col='Object') # if unsure of which regions to select, I just process the full image then crop the files later - this could be made more efficient by cropping the image before processing


In [ ]:
ROI_upper_left = (0, 0)
ROI_lower_right = (1000,1000)
roi_crop, plot_crop = crop_roi(segmentation_mask, plot_segmentation, ROI_upper_left, ROI_lower_right)

In [ ]:
background_color = 'black'
colors_list = [background_color] + [phenotype_color_map[identifier] for identifier in unique_identifiers] 
colors_list

In [ ]:
background_color = 'black'
colors_list = [background_color] + [neighborhood_color_map[identifier] for identifier in unique_identifiers] 
colors_list

In [ ]:
final_cmap = colors.ListedColormap(colors_list)
final_cmap

In [ ]:
create_plot(roi_crop=roi_crop, plot_crop=plot_crop, cmap=final_cmap, boundaries=True, boundary_color=(0, 0, 0, 1), boundary_thickness=1,
            save_path=os.path.join(save_path, f'{sample.split(".csv")[0]}_{target_col}.{extension}')
            )